In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
import json

# Bước 1: Đọc dữ liệu train
train_file = '/kaggle/input/otto-recommender-system/train.jsonl'
test_file = '/kaggle/input/otto-recommender-system/test.jsonl'

print("Đang đọc và xử lý dữ liệu train...")
sessions = []
products = []
interactions = []

# Đọc dữ liệu và chỉ sử dụng 100,000 sessions đầu tiên
max_sessions = 1000
session_count = 0

with open(train_file, 'r') as file:
    for line in file:
        if session_count >= max_sessions:
            break
        data = json.loads(line)
        session_id = data['session']
        session_count += 1
        for event in data['events']:
            aid = event['aid']
            event_type = event['type']
            weight = 1
            if event_type == 'carts':
                weight = 3
            elif event_type == 'orders':
                weight = 5
            sessions.append(session_id)
            products.append(aid)
            interactions.append(weight)

interaction_df = pd.DataFrame({'session': sessions, 'product': products, 'interaction': interactions})
print(f"Đã xử lý {session_count} sessions.")

# Lọc chỉ giữ lại top 2,000 sản phẩm phổ biến nhất
top_products = interaction_df['product'].value_counts().index[:20]
interaction_df_filtered = interaction_df[interaction_df['product'].isin(top_products)]

# Tạo ma trận sparse từ dữ liệu đã giảm kích thước
interaction_matrix = interaction_df_filtered.pivot_table(index='session', columns='product', values='interaction', fill_value=0)
interaction_sparse = csr_matrix(interaction_matrix)

print(f"Kích thước ma trận sau lọc: {interaction_sparse.shape}")

# Bước 2: Áp dụng SVD
print("Đang áp dụng SVD...")
svd = TruncatedSVD(n_components=2, random_state=42)  # Giảm số yếu tố ẩn
latent_matrix = svd.fit_transform(interaction_sparse)

# Tái tạo ma trận dự đoán
reconstructed_matrix = np.dot(latent_matrix, svd.components_)
reconstructed_df = pd.DataFrame(reconstructed_matrix, index=interaction_matrix.index, columns=interaction_matrix.columns)
print("Ma trận dự đoán đã được tạo.")

# Bước 3: Dự đoán cho mỗi session trong tập test
print("Đang đọc và xử lý dữ liệu test...")
predictions = []

with open(test_file, 'r') as file:
    for line in file:
        data = json.loads(line)
        session_id = data['session']
        if session_id in reconstructed_df.index:
            # Lấy top 20 sản phẩm dự đoán
            predicted_products = reconstructed_df.loc[session_id].sort_values(ascending=False).index[:20]
        else:
            predicted_products = interaction_df['product'].value_counts().index[:20]
        
        # Tạo dự đoán cho các loại hành động
        predictions.append(f"{session_id}_clicks," + " ".join(map(str, predicted_products)))
        predictions.append(f"{session_id}_carts," + " ".join(map(str, predicted_products)))
        predictions.append(f"{session_id}_orders," + " ".join(map(str, predicted_products)))

# Bước 4: Ghi kết quả ra file submission.csv
output_file = '/kaggle/working/submission.csv'
print(f"Đang ghi kết quả ra file {output_file}...")

with open(output_file, 'w') as f:
    f.write("session_type,labels\n")
    for line in predictions:
        f.write(line + "\n")

print("Dự đoán hoàn tất! File submission.csv đã được tạo.")


## Method